In [207]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [222]:
response = requests.get("https://poemlove.co.kr/bbs/board.php?bo_table=tb01&wr_id=271686")
print(response)
# 웹페이지의 내용을 성공적으로 가져온 경우
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "lxml")
    fail_message = soup.find("meta", {"name": 'title'})
    if "오류안내 페이지" in fail_message["content"]:
        print("fail")
    headline = soup.find("h1", {"itemprop": "headline"}).get_text().strip() # 시 제목
    author_box = soup.find_all("div", {"class": "panel panel-default view-head"})
    print(author_box[-1])
    all_text = author_box[-1].find_all("strong")[0].get_text()
    print(all_text)               
    # author = author_box.find("span", {"class": "member"}).get_text()        # 시 저자
    # poem = soup.find("div", {'itemprop': 'description', 'class': 'view-content'}).get_text(separator="\n")
    # print(headline)
    # print(author)
    # print(poem)

<Response [200]>
<div class="panel panel-default view-head">
<div class="panel-heading">
<font color="#7A8FDB">저자</font> : <strong>류인순</strong>
				
								    <font color="#7A8FDB">시집명</font> : <strong></strong>
<font color="#7A8FDB">출판(발표)연도</font> : <strong>2024.03.05.</strong>
				
								    <font color="#7A8FDB">출판사</font> : <strong></strong>
</div>
</div>
류인순


In [ ]:
# '\n\t\t\t\t햇살의 기쁨\r\n \n
# '\n\t\t\t\t매화\r\n \n\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 글 : 박동수\r\n \n
# '\n\t\t\t\t내가 먼저/ 홍수희\r\n \n \n
# '\n\t\t\t\t풍경 ( 風 磬 )\r\n\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 - 다서 신형식\r\n \n
# '\n\t\t\t\t홍매화 향기처럼 / 정심 김덕성\r\n \n \n
# '\n\t\t\t\t때로는 꽃들이 마음에 쉼표가 -> 처음으로 \r을 만날 때
# '\n\t\t\t\t아지랑이\r\n소산/문 재학\r\n \n
# '\n\t\t\t\t그래서 순수만이 만져볼 수 있는
# '\n\t\t\t\t너에게 쓰는 편지\r\n \n
# '\n\t\t\t\t문신 / 이재봉\r\n \n
# "\n\t\t\t\t삼일절 \r\n \n靑山 손병흥 \r\n \n
# '\n\t\t\t\t5월 들길 / 성백군\r\n \n \n늙은 봄과 젊은 여름이\r\n공생하는 5월 들길을 걷는다\r\n \n

# 1. \r\n \n 가 1번 나오는 경우 / 2번 나오는 경우
# 2. \r\n \n \n
# 3. \r\n \n \n이 나오고 내용이 나오지만 그 뒤에 \r\n \n이 나오는 경우 -> 2,3의 경우에는 이 패턴이 나오면 끝내야 함
# 4. 아예 없는 경우

# 제목과 작가를 strip(제목 or 작가)로 지우고, 나머지 특수기호들을 strip으로 싹 지워준다면 가장 깔끔할 것 같다
# strip()을 사용하니 양끝에서 일반문자를 만날 때까지 특수기호를 제거한다. 

In [173]:
poem

'\n\t\t\t\t풍경 ( 風 磬 )\r\n\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 - 다서 신형식\r\n \n꽃향기 냄새가\r\n풍경 ( 風磬 )앞에서 맑다\r\n은은하다 못해 찬란하다\r\n난 아직도 나를 몰라\r\n바람에 흔들리고\r\n나를 깨우는 그대 때문에\r\n혼자 일어나\r\n조용한 새벽을 준비한다\t\t\t'

In [178]:
# 시의 내용만 가져오기 위한 전처리
def preprocessing(headline: str, author: str, poem: str) -> str:
    data = poem.replace("\r", "")                           # 특수문자 제거
    data = data.replace("\n \n", "\n\n")                    # 연 구분
    if data.find(author) > 0:
        data = data[data.find(author) + len(author):]       # 작가 제거
    data = data.replace(headline, "", 1).replace("/", "")   # 제목 및 / 제거
    data = re.sub(r'[^가-힣0-9\n\s,.]', '', data)           # 외국어 및 특수문자 제거
    data = data.strip()                                     # 양옆 \기호 제거

    return data

In [179]:
preprocessed_poem = preprocessing(headline, author, poem)
preprocessed_poem

'다서 신형식\n\n꽃향기 냄새가\n풍경   앞에서 맑다\n은은하다 못해 찬란하다\n난 아직도 나를 몰라\n바람에 흔들리고\n나를 깨우는 그대 때문에\n혼자 일어나\n조용한 새벽을 준비한다'

In [ ]:
# url request -> html속에서 시 추출 -> 시 전처리 -> csv 파일에 저장

In [182]:
data_file = {"title":[],
            "author": [],
            "poem": []}

In [184]:
text = ["\n\t\t\t\t햇살의 기쁨\r\n \n",
"\n\t\t\t\t매화\r\n \n\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 글 : 박동수\r\n \n",
"\n\t\t\t\t내가 먼저/ 홍수희\r\n \n \n",
"\n\t\t\t\t풍경 ( 風 磬 )\r\n\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 - 다서 신형식\r\n \n",
"\n\t\t\t\t홍매화 향기처럼 / 정심 김덕성\r\n \n \n"]

for i in text:
    data_file["title"].append("A")
    data_file["author"].append("B")
    data_file["poem"].append(i)

df = pd.DataFrame(data_file)
df.to_csv("df.csv", index=False)